In [29]:
import os
import random
from utils.hand_model import HandModel
import numpy as np
import transforms3d
import torch
import trimesh


In [30]:
mesh_path = "/home/arjun/datasets/dexgraspnet/meshdata"
data_path = "/home/arjun/dexlite/DexGraspNet/data/experiments/exp/results"


use_visual_mesh = True

hand_file = "mjcf/shadow_hand_vis.xml" 
# "mjcf/shadow_hand_wrist_free.xml"

joint_names = [
    'robot0:FFJ3', 'robot0:FFJ2', 'robot0:FFJ1', 'robot0:FFJ0',
    'robot0:MFJ3', 'robot0:MFJ2', 'robot0:MFJ1', 'robot0:MFJ0',
    'robot0:RFJ3', 'robot0:RFJ2', 'robot0:RFJ1', 'robot0:RFJ0',
    'robot0:LFJ4', 'robot0:LFJ3', 'robot0:LFJ2', 'robot0:LFJ1', 'robot0:LFJ0',
    'robot0:THJ4', 'robot0:THJ3', 'robot0:THJ2', 'robot0:THJ1', 'robot0:THJ0'
]
translation_names = ['WRJTx', 'WRJTy', 'WRJTz']
rot_names = ['WRJRx', 'WRJRy', 'WRJRz']


In [31]:
hand_model = HandModel(
    mjcf_path='mjcf/shadow_hand_wrist_free.xml',
    mesh_path='mjcf/meshes',
    contact_points_path='mjcf/contact_points.json',
    penetration_points_path='mjcf/penetration_points.json'
)


In [32]:
grasp_code_list = []
for code in os.listdir(data_path):
    grasp_code_list.append(code[:-4])


In [33]:
grasp_code = random.choice(grasp_code_list)
# grasp_code="sem-Car-58379002fbdaf20e61a47cff24512a0"
grasp_data = np.load(
    os.path.join(data_path, grasp_code+".npy"), allow_pickle=True)
object_mesh_origin = trimesh.load(os.path.join(
    mesh_path, grasp_code, "coacd/decomposed.obj"))
print(grasp_code)


sem-Car-2f28e2bd754977da8cfac9da0ff28f62


In [34]:
index = random.randint(0, len(grasp_data) - 1)
print(index)


qpos = grasp_data[index]['qpos']
rot = np.array(transforms3d.euler.euler2mat(
    *[qpos[name] for name in rot_names]))
rot = rot[:, :2].T.ravel().tolist()
hand_pose = torch.tensor([qpos[name] for name in translation_names] + rot + [qpos[name]
                         for name in joint_names], dtype=torch.float, device="cpu").unsqueeze(0)
hand_model.set_parameters(hand_pose)
hand_mesh = hand_model.get_trimesh_data(0)
object_mesh = object_mesh_origin.copy().apply_scale(grasp_data[index]["scale"])


99


In [35]:
# Create visualization with origin, axes, and table plane
scene = trimesh.Scene()

# Add hand and object meshes
scene.add_geometry(hand_mesh, geom_name='hand')
scene.add_geometry(object_mesh, geom_name='object')

# Add coordinate axes at origin (RGB = XYZ)
axis_length = 0.1
axes = trimesh.creation.axis(origin_size=0.01, axis_length=axis_length)
scene.add_geometry(axes, geom_name='axes')

# Add table plane (at z = bottom of object)
table_z = object_mesh.bounds[0, 2]  # bottom of scaled object
table_size = 0.3
table_plane = trimesh.creation.box(extents=[table_size, table_size, 0.002])
table_plane.apply_translation([0, 0, table_z - 0.001])  # just below object bottom
table_plane.visual.face_colors = [150, 150, 150, 180]  # gray, semi-transparent
scene.add_geometry(table_plane, geom_name='table')

# Visualize ALL mesh vertices (same as E_table calculation in energy.py)
all_verts = []
for link_name in hand_model.mesh:
    link_verts = hand_model.mesh[link_name]['vertices']  # (n_verts, 3) in link frame
    # Transform to world frame using current FK status
    transformed = hand_model.current_status[link_name].transform_points(link_verts)
    if len(transformed.shape) == 2:
        transformed = transformed.unsqueeze(0)
    # Apply global rotation and translation
    transformed = transformed @ hand_model.global_rotation.transpose(1, 2) + hand_model.global_translation.unsqueeze(1)
    all_verts.append(transformed[0].detach().cpu().numpy())

all_pts = np.vstack(all_verts)

# Color points: red if below table, green if above
colors = np.zeros((len(all_pts), 4), dtype=np.uint8)
below_mask = all_pts[:, 2] < table_z
colors[below_mask] = [255, 0, 0, 255]      # red = below table (BAD)
colors[~below_mask] = [0, 255, 0, 255]     # green = above table (OK)

# Create point cloud
pts_cloud = trimesh.PointCloud(all_pts, colors=colors)
scene.add_geometry(pts_cloud, geom_name='E_table_points')

# Print stats
n_below = below_mask.sum()
print(f"Hand mesh vertices: {len(all_pts)} total, {n_below} below table (red), {len(all_pts)-n_below} above (green)")
print(f"Table Z: {table_z:.4f}")
print(f"Min hand vertex Z: {all_pts[:, 2].min():.4f}")
# print(hand_model.global_rotation)
# Show the scene
scene.show()

Hand mesh vertices: 30990 total, 1958 below table (red), 29032 above (green)
Table Z: -0.0278
Min hand vertex Z: -0.0406
